In [2]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import precision_score

import urllib2
import bs4
import time
import operator
import re 
    
%matplotlib inline
import seaborn as sns

In [3]:
#Time Series Data
UR = pd.read_csv('UnempRate.csv', sep=',', na_values=[" ", ""])
crime = pd.read_csv('violent_crime_count_age17andup.csv', sep=',', na_values=[" ", ""])
#crime.iloc[:,1:]/population.iloc[:,1:] 
#but missing 2016 data, so 2016 data not used
#prescription is per 100
prescription = pd.read_csv('prescription_rate.csv', sep=',', na_values=[" ", ""])
population = pd.read_csv('population.csv', sep=',', na_values=[" ", ""])
birth_rate = pd.read_csv('birth_rate.csv', sep=',', na_values=[" ", ""])
death_rate = pd.read_csv('death_rate.csv', sep=',', na_values=[" ", ""])

In [6]:
#Normalizing Time-Series Data
UR.iloc[:,1:]=UR.iloc[:,1:]/100
birth_rate.iloc[:,1:]=birth_rate.iloc[:,1:]/1000
death_rate.iloc[:,1:]=death_rate.iloc[:,1:]/1000
prescription.iloc[:,1:]=prescription.iloc[:,1:]/100

In [7]:
#Non-Time Series Data
data = pd.read_csv('cleaned_GaCountiesODDeath.csv', na_values=[" ", ""] ,sep=',', index_col=0).sort('County').reset_index(drop=True)
hospital = pd.read_csv('cleaned_GaHospitals.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)
dropbox = pd.read_csv('drug_dropbox.csv', na_values=[" ", ""] ,sep=',').fillna(0)
ems = pd.read_csv('cleaned_ems.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)
rehab = pd.read_csv('cleaned_rehab.csv', na_values=[" ", ""] ,sep=',', index_col=0).fillna(0)

hospital=hospital.rename(columns = {'Count':'hospital_count'})
dropbox=dropbox.rename(columns = {'COUNT':'dropbox_count'})
dropbox=dropbox.rename(columns = {'24Availability':'dropbox_247availability'})

In [8]:
normalized_data = data.iloc[:,1:]/population.iloc[:,1:]

In [9]:
df = data[['County','Rural']].copy()
df1 = pd.concat([df, rehab['Rehab_Count'],  ems['EMS_COUNT'],
                 hospital['hospital_count'], dropbox['dropbox_247availability'], dropbox['dropbox_count']], axis=1)

In [10]:
#data[['Counties','Rural', '2016', '1yrlag', '2yrlag', '10yrlag']]
df1['1yrlag']=(data['2016']-data['2015'])/data['2015']
df1['5yrlag']=(data['2016']-data['2011'])/data['2011']
df1['10yrlag']=(data['2016']-data['2006'])/data['2006']

df1['UR_1yrlag']=(UR['2016']-UR['2015'])/UR['2015']
df1['UR_5yrlag']=(UR['2016']-UR['2011'])/UR['2011']
df1['UR_10yrlag']=(UR['2016']-UR['2006'])/UR['2006']

df1['CR_1yrlag']=(crime['2015']-crime['2014'])/crime['2014']
df1['CR_5yrlag']=(crime['2015']-crime['2010'])/crime['2010']
df1['CR_10yrlag']=(crime['2015']-crime['2005'])/crime['2005']

df1['PR_1yrlag']=(prescription['2016']-prescription['2015'])/prescription['2015']
df1['PR_5yrlag']=(prescription['2016']-prescription['2011'])/prescription['2011']
df1['PR_10yrlag']=(prescription['2016']-prescription['2006'])/prescription['2006']

df1['BR_1yrlag']=(birth_rate['2016']-birth_rate['2015'])/birth_rate['2015']
df1['BR_5yrlag']=(birth_rate['2016']-birth_rate['2011'])/birth_rate['2011']
df1['BR_10yrlag']=(birth_rate['2016']-birth_rate['2006'])/birth_rate['2006']

df1['DR_1yrlag']=(death_rate['2016']-death_rate['2015'])/death_rate['2015']
df1['DR_5yrlag']=(death_rate['2016']-death_rate['2011'])/death_rate['2011']
df1['DR_10yrlag']=(death_rate['2016']-death_rate['2006'])/death_rate['2006']

df1=df1.replace([np.inf, -np.inf], np.nan)
df1 = df1.fillna(0)

death_rate_2016=death_rate.rename(columns = {'2016':'2016_death_rate'})
birth_rate_2016=birth_rate.rename(columns = {'2016':'2016_birth_rate'})
prescription_2016=prescription.rename(columns = {'2016':'2016_prescription'})

df2 = pd.concat([df1, death_rate_2016['2016_death_rate'], birth_rate_2016['2016_birth_rate'], prescription_2016['2016_prescription'], normalized_data['2016']], axis=1)
df2=df2.rename(columns = {'2016':'2016_ODRate'})

In [11]:
df2.columns
nat_OD_rate=0.00016219127

In [12]:
#2015 National OD Rate
#52,404/323100000
df2['2016ODabovenatavg'] = df2['2016_ODRate'] > nat_OD_rate

In [13]:
df2.to_csv('AggredgatedData.csv')

In [169]:
features = list(df2.columns[1:-1])
X = df2[features]
Y = df2['2016_ODRate']

In [170]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [171]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)

In [172]:
Y_pred = clf.predict(X_test)

In [177]:
Y_test

78     0.000000
155    0.001256
128    0.000197
55     0.000018
94     0.000072
29     0.000000
147    0.000839
51     0.000157
98     0.000000
141    0.000747
19     0.000019
60     0.000135
15     0.000013
65     0.000235
24     0.000017
30     0.000007
126    0.000350
101    0.000037
96     0.000173
16     0.000000
151    0.004232
18     0.000158
12     0.000218
9      0.000053
31     0.000146
125    0.001173
95     0.000149
56     0.000083
145    0.000250
152    0.000376
135    0.001879
76     0.000000
75     0.000007
138    0.003248
2      0.000000
86     0.000000
45     0.000073
42     0.000075
68     0.000000
118    0.001027
26     0.000161
137    0.001103
146    0.000155
90     0.000000
66     0.000000
36     0.000000
82     0.000526
22     0.000015
85     0.000000
81     0.000000
112    0.000261
11     0.000308
109    0.000083
Name: 2016_ODRate, dtype: float64